# Testing EUGENE prediction

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing predictions with EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li>Config for EUGENES</li></b>
    </ul>
</div>

In [3]:
import os
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [2]:
import eugene as eu

# Predictions with a EUGENE model


## Load data

In [4]:
sdata = eu.datasets.random1000()

In [5]:
eugene = eu.models.DeepBind(input_len=66)

In [18]:
eu.predict.predictions(eugene, sdata=sdata, out_dir="../_out/test_", save_preds="random1000_PREDS")

Global seed set to 13
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Predicting: 0it [00:00, ?it/s]

[['seq001' '-0.3775099' '1.0']
 ['seq002' '-0.37696442' '0.0']
 ['seq003' '-0.3778545' '1.0']
 ...
 ['seq998' '-0.3799876' '0.0']
 ['seq999' '-0.37663758' '0.0']
 ['seq1000' '-0.3763308' '0.0']]


In [20]:
sdata.seqs_annot

,TARGETS,random1000_PREDS
seq001,1.0,-0.377510
seq002,0.0,-0.376964
seq003,1.0,-0.377854
seq004,0.0,-0.377361
seq005,0.0,-0.377899
...,...,...
seq996,0.0,-0.377196
seq997,0.0,-0.376374
seq998,0.0,-0.379988
seq999,0.0,-0.376638


---

# Scratch